<a href="https://colab.research.google.com/github/pulkit8690/Topsis-For-Pretrained-Model-Text-Classification-/blob/main/Parameter_Evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets

In [ ]:
import pandas as pd
import time
from transformers import pipeline
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, average_precision_score, matthews_corrcoef, cohen_kappa_score,log_loss
from datasets import load_dataset
import matplotlib.pyplot as plt
import seaborn as sns


In [ ]:
models = [
    "Sakil/IMDB_URDUSENTIMENT_MODEL",
    "mnoukhov/gpt2-imdb-sentiment-classifier",
    "AdapterHub/bert-base-uncased-pf-imdb",
    "wrmurray/roberta-base-finetuned-imdb",
    "kurianbenoy/distilbert-base-uncased-finetuned-sst-2-english-finetuned-imdb",

]

In [ ]:
models

In [ ]:
dataset = load_dataset('imdb',streaming=True)

In [ ]:
data = [item  for item in dataset['test'] if len(item['text']) <512]
len(data)

In [ ]:
df = pd.DataFrame(data)

In [ ]:
NUM_ROWS = 2000

In [ ]:
newDf = pd.concat([df[df.label == 0].head(NUM_ROWS //2),df[df.label ==1].head(NUM_ROWS//2)]).sample(frac = 1).reset_index(drop = True)


In [ ]:
newDf.head()

In [ ]:
texts = newDf.text.tolist()
labels = newDf.label.tolist()

In [ ]:
sns.countplot(x = labels)

In [ ]:
FORMAT_LABELS ={
    'Label_0':0,
    "Label_1":1,
    "POSITIVE":1,
    "NEGATIVE":0,
    'neg':0,
    'pos':1
}

In [ ]:
def evaluate_model(model_name):

    print('model initialized')
    pipe = pipeline("text-classification", model=model_name)
    start_time = time.time()
    res = pipe(texts)
    end_time = time.time()

    predicted_labels = list(map(lambda x: FORMAT_LABELS.get(x['label'],0),res))
    probs = [ item['score'] for item in res ]
    print('Calculate evaluation metrics')
    # Calculate evaluation metrics
    accuracy = accuracy_score(labels, predicted_labels)
    precision = precision_score(labels, predicted_labels, average="weighted")
    recall = recall_score(labels, predicted_labels, average="weighted")
    f1 = f1_score(labels, predicted_labels, average="weighted")
    roc_auc = roc_auc_score(labels, predicted_labels)
    avg_precision = average_precision_score(labels, predicted_labels)
    mcc = matthews_corrcoef(labels, predicted_labels)
    kappa = cohen_kappa_score(labels, predicted_labels)
    logloss = log_loss(labels,probs)

    # Calculate training time
    training_time = end_time - start_time

    return {
        "Model": model_name,
        "Accuracy": accuracy,
        "Precision": precision,
        "Recall": recall,
        "F1 Score": f1,
        "ROC-AUC": roc_auc,
        "Average Precision": avg_precision,
        "Matthews Correlation Coefficient": mcc,
        "Cohen's Kappa": kappa,
        "Time (s)": training_time,
        'Log Loss': logloss,
    }

In [ ]:
results = []

for model_name in models:
        try:
            result = evaluate_model(model_name)
            results.append(result)
            print("Done",model_name)
        except:
            print(model_name)

# making a dataframe
df = pd.DataFrame(results)

In [ ]:
df.head()

In [ ]:
df.to_csv("Inputdata.csv")